In [1]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
total_valuation = 0    
profit = 0
commission = 5
max_profit = 0
min_loss = 0

C:\Users\Bikal\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


In [3]:
symbols = """
select distinct(name) from stockdatadaily order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [4]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [5]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = 0.0
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        self.training = list()

sql1= """
select distinct(name) from stockdatadaily order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)



<h1> Updating resources

In [6]:
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdatadaily where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done
100 % done

In [7]:
sql_dates = """
select distinct(date) from stockdatadaily order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Trading strategy with the training sets generation

In [8]:
def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global total_valuation
    global list_for_Risk
    global capital
    global buy_times
    global max_profit
    global min_loss
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdatadaily where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        buy_stocks(current_price, current_symbol, date)
                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                
                                win += 1
                                current_symbol.flag_for_wins = True
                                
                        
                        
                    if current_symbol.stop_order > current_price:                
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        if profit_today > 0:
                            
                            if current_symbol.flag_for_wins == False:
                                win += 1
                                
                            current_symbol.flag_for_wins = False
                            
                        else:
                            loss += 1
                            
                            current_symbol.flag_for_wins = False
                    
                            
                        profit += profit_today
                        
                        if profit > max_profit:
                            max_profit = profit
                        if profit < min_loss:
                            min_loss = profit
                        
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("Profit",profit)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [ ]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def buy_stocks(current_price, current_symbol, date):
    global position
    global remaining_cash
    global commission
    
    quantity = (0.02 * remaining_cash) // (current_price - (current_symbol.support * 0.97))

    if quantity > 1:
        current_symbol.bought = current_price
        current_symbol.stop_order = current_symbol.support * 0.97
        remaining_cash = remaining_cash - commission - (quantity * current_price)
        position[current_symbol.symbol] = quantity
        print("Buying ", quantity, "stocks of ", current_symbol.symbol, "At price", current_symbol.bought, "on date", date)
    else:
        pass
    
def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data
   

In [ ]:
automatic_trader(symbol_list)

Buying  127.0 stocks of  AB At price 30.0 on date 2000-01-05
Buying  47.0 stocks of  ABC At price 16.19 on date 2000-01-05
Buying  276.0 stocks of  AEY At price 3.06 on date 2000-01-05
Buying  42.0 stocks of  AKS At price 18.88 on date 2000-01-05
Buying  38.0 stocks of  ALE At price 17.56 on date 2000-01-05
Buying  62.0 stocks of  AP At price 10.69 on date 2000-01-05
Buying  5.0 stocks of  ARNC At price 86.5 on date 2000-01-05
Buying  26.0 stocks of  ATO At price 20.5 on date 2000-01-05
Buying  23.0 stocks of  AVA At price 15.44 on date 2000-01-05
Buying  2.0 stocks of  AVY At price 73.88 on date 2000-01-05
Buying  3.0 stocks of  BA At price 43.31 on date 2000-01-05
Buying  14.0 stocks of  BAM At price 13.69 on date 2000-01-05
Buying  9.0 stocks of  BFS At price 14.81 on date 2000-01-05
Buying  3.0 stocks of  CAKE At price 34.31 on date 2000-01-05
Buying  2.0 stocks of  CAT At price 49.88 on date 2000-01-05
Buying  2.0 stocks of  CENX At price 15.94 on date 2000-01-05
Buying  10.0 stoc

Buying  2.0 stocks of  EFOI At price 0.925 on date 2000-03-02
Buying  2.0 stocks of  PHX At price 7.38 on date 2000-03-02
Selling  47.0 stocks of ABC  at price 13.12 on date 2000-03-08
Buying  9.0 stocks of  ACHV At price 4.25 on date 2000-03-08
Buying  3.0 stocks of  ACXM At price 27.0 on date 2000-03-08
Buying  3.0 stocks of  AE At price 12.63 on date 2000-03-08
Buying  5.0 stocks of  AXAS At price 3.0 on date 2000-03-08
Buying  2.0 stocks of  BHE At price 36.0 on date 2000-03-08
Buying  3.0 stocks of  BLK At price 18.88 on date 2000-03-08
Buying  2.0 stocks of  CBL At price 22.75 on date 2000-03-08
Buying  2.0 stocks of  CCE At price 22.19 on date 2000-03-08
Selling  4.0 stocks of CMO  at price 3.81 on date 2000-03-08
Buying  2.0 stocks of  CTHR At price 8.625 on date 2000-03-08
Buying  2.0 stocks of  CVA At price 6.375 on date 2000-03-08
Buying  2.0 stocks of  EMITF At price 11.38 on date 2000-03-08
Buying  2.0 stocks of  ESE At price 15.0 on date 2000-03-08
Selling  2.0 stocks of 

Selling  2.0 stocks of EFOI  at price 0.4449 on date 2001-06-29
Buying  2.0 stocks of  EGY At price 1.12 on date 2001-06-29
Buying  2.0 stocks of  MBVX At price 9.51 on date 2001-06-29
Buying  2.0 stocks of  OLP At price 13.57 on date 2001-06-29
Selling  2.0 stocks of GIII  at price 9.6 on date 2001-07-18
Selling  2.0 stocks of MNST  at price 3.8 on date 2001-07-19
Selling  3.0 stocks of ACXM  at price 11.99 on date 2001-08-02
Buying  2.0 stocks of  TCO At price 13.3 on date 2001-08-02
Selling  2.0 stocks of DGSE  at price 4.15 on date 2001-08-16
Selling  7.0 stocks of VTNR  at price 0.01 on date 2001-08-16
Buying  2.0 stocks of  VKI At price 12.86 on date 2001-09-04
Selling  2.0 stocks of CASM  at price 0.45 on date 2001-09-20
Selling  2.0 stocks of MBVX  at price 8.05 on date 2001-09-24
Selling  3.0 stocks of MCF  at price 2.95 on date 2001-09-24
Selling  2.0 stocks of SLP  at price 1.2 on date 2001-09-24
Selling  2.0 stocks of CET  at price 24.4 on date 2001-10-25
Buying  3.0 stocks

Buying  3.0 stocks of  HIL At price 5.09 on date 2004-12-20
Selling  2.0 stocks of SVRA  at price 1.12 on date 2005-01-13
Selling  4.0 stocks of FLXS  at price 16.62 on date 2005-03-22
Buying  2.0 stocks of  MICT At price 1.75 on date 2005-03-22
Buying  4.0 stocks of  BIOC At price 3.15 on date 2005-03-23
Selling  2.0 stocks of MIN  at price 6.4 on date 2005-04-08
Selling  4.0 stocks of CRMT  at price 21.98 on date 2005-04-25
Buying  2.0 stocks of  KMM At price 10.19 on date 2005-04-25
Buying  2.0 stocks of  TRNS At price 4.19 on date 2005-04-26
Selling  2.0 stocks of CEMI  at price 0.8 on date 2005-04-28
Buying  16.0 stocks of  SVM At price 1.31 on date 2005-05-04
Selling  6.0 stocks of AXL  at price 18.9 on date 2005-05-16
Buying  2.0 stocks of  DLR At price 15.13 on date 2005-05-16
Buying  2.0 stocks of  TXMD At price 1.5 on date 2005-05-16
Buying  4.0 stocks of  GMO At price 1.65 on date 2005-05-17
Selling  10.0 stocks of BKT  at price 6.62 on date 2005-06-21
Buying  2.0 stocks of 

Buying  2.0 stocks of  RJA At price 10.81 on date 2007-12-13
Buying  2.0 stocks of  CATM At price 10.03 on date 2007-12-21
Selling  2.0 stocks of IART  at price 42.67 on date 2007-12-26
Selling  2.0 stocks of ASX  at price 4.91 on date 2007-12-27
Selling  6.0 stocks of CXE  at price 5.28 on date 2007-12-27
Buying  2.0 stocks of  FNJN At price 4.3 on date 2007-12-27
Buying  2.0 stocks of  PYS At price 20.19 on date 2008-01-02
Selling  2.0 stocks of CATM  at price 9.2 on date 2008-01-08
Selling  2.0 stocks of PKB  at price 16.65 on date 2008-01-09
Selling  2.0 stocks of EWBC  at price 22.25 on date 2008-01-22
Selling  2.0 stocks of PXSV  at price 14.49 on date 2008-01-22
Selling  2.0 stocks of TRIL  at price 0.27 on date 2008-01-22
Buying  2.0 stocks of  WSCI At price 8.7 on date 2008-01-22
Buying  9.0 stocks of  ARR At price 9.2 on date 2008-01-23
Selling  5.0 stocks of PFIS  at price 24.0 on date 2008-01-28
Buying  2.0 stocks of  ATAI At price 10.3 on date 2008-01-31
Selling  2.0 stock

Selling  2.0 stocks of MNP  at price 10.21 on date 2009-01-05
Selling  7.0 stocks of PLM  at price 0.87 on date 2009-01-05
Selling  2.0 stocks of ATAI  at price 5.0 on date 2009-01-06
Selling  4.0 stocks of BIOC  at price 3.06 on date 2009-01-12
Selling  5.0 stocks of HTM  at price 1.09 on date 2009-01-12
Selling  2.0 stocks of PSJ  at price 13.18 on date 2009-01-12
Selling  3.0 stocks of RJN  at price 4.94 on date 2009-01-12
Selling  3.0 stocks of UBA  at price 14.04 on date 2009-01-13
Selling  2.0 stocks of CCE  at price 11.72 on date 2009-01-20
Selling  3.0 stocks of ZSL  at price 16.95 on date 2009-01-22
Selling  2.0 stocks of RDIB  at price 3.71 on date 2009-01-23
Buying  10.0 stocks of  JE At price 8.38 on date 2009-02-06
Buying  4.0 stocks of  IGOV At price 95.9205 on date 2009-02-09
Selling  3.0 stocks of MSN  at price 0.5499 on date 2009-02-11
Selling  18.0 stocks of BSRR  at price 7.56 on date 2009-02-24
Buying  17.0 stocks of  HBM At price 4.4 on date 2009-02-24
Buying  2.0 

Selling  2.0 stocks of CLH  at price 55.67 on date 2011-07-27
Buying  2.0 stocks of  EBND At price 31.47 on date 2011-07-27
Buying  3.0 stocks of  BLRX At price 5.24 on date 2011-07-29
Selling  4.0 stocks of GCVRZ  at price 1.07 on date 2011-08-08
Selling  7.0 stocks of GXG  at price 18.96 on date 2011-08-08
Buying  3.0 stocks of  INN At price 10.45 on date 2011-08-08
Buying  4.0 stocks of  PFIE At price 1.0 on date 2011-08-08
Buying  2.0 stocks of  TUZ At price 51.22 on date 2011-08-08
Selling  2.0 stocks of TNA  at price 48.98 on date 2011-08-15
Buying  2.0 stocks of  ASEA At price 16.5292 on date 2011-08-16
Selling  3.0 stocks of INN  at price 7.47 on date 2011-08-19
Selling  2.0 stocks of REN  at price 13.01 on date 2011-09-02
Buying  2.0 stocks of  TDJ At price 26.03 on date 2011-09-07
Selling  2.0 stocks of GRID  at price 26.0 on date 2011-09-09
Buying  5.0 stocks of  SGBX At price 0.59 on date 2011-09-15
Buying  10.0 stocks of  ACST At price 1.5102 on date 2011-09-16
Selling  2.

Buying  25.0 stocks of  WKHS At price 0.125 on date 2014-08-22
Selling  2.0 stocks of OSBCP  at price 10.03 on date 2014-09-11
Selling  2.0 stocks of BTG  at price 2.15 on date 2014-09-18
Selling  2.0 stocks of MLAB  at price 65.78 on date 2014-09-18
Buying  2.0 stocks of  NAVI At price 17.96 on date 2014-09-18
Buying  2.0 stocks of  WBID At price 25.34 on date 2014-09-18
Selling  2.0 stocks of MICT  at price 3.495 on date 2014-09-22
Selling  2.0 stocks of FKU  at price 38.64 on date 2014-10-01
Buying  2.0 stocks of  CFG At price 23.59 on date 2014-10-02
Buying  5.0 stocks of  NXEO At price 10.0 on date 2014-10-03
Buying  2.0 stocks of  DWAT At price 9.9699 on date 2014-10-06
Selling  2.0 stocks of MLPY  at price 17.4714 on date 2014-10-13
Selling  2.0 stocks of SNGX  at price 1.66 on date 2014-10-13
Selling  2.0 stocks of SRCI  at price 12.25 on date 2014-11-11
Selling  2.0 stocks of DISCK  at price 31.736 on date 2014-11-12
Buying  2.0 stocks of  SCTO At price 25.61 on date 2014-11-1

Selling  2.0 stocks of SSNC  at price 58.365 on date 2016-06-17
Buying  3.0 stocks of  AUG At price 2.16 on date 2016-06-20
Buying  2.0 stocks of  BEMO At price 25.37 on date 2016-06-20
Buying  3.0 stocks of  EHT At price 10.16 on date 2016-06-20
Selling  3.0 stocks of CLDC  at price 5.575 on date 2016-07-13
Buying  18.0 stocks of  CRON At price 0.2749 on date 2016-07-20
Selling  6.0 stocks of PTEU  at price 22.76 on date 2016-07-20
Buying  2.0 stocks of  PZT At price 25.69 on date 2016-07-20
Buying  2.0 stocks of  QINC At price 22.7699 on date 2016-07-20
Selling  3.0 stocks of ALGT  at price 143.54 on date 2016-07-25
Buying  2.0 stocks of  BKHU At price 70.0 on date 2016-07-25
Buying  3.0 stocks of  BRX At price 27.69 on date 2016-07-25
Buying  2.0 stocks of  BSCM At price 21.639 on date 2016-07-25
Buying  3.0 stocks of  CEMI At price 6.2 on date 2016-07-25
Buying  2.0 stocks of  CHSCM At price 28.87 on date 2016-07-25
Buying  2.0 stocks of  DGRO At price 28.22 on date 2016-07-25
Buyi

In [ ]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)